In [ ]:
try:
    import crewai
    print("CrewAI is installed! Version:", crewai.__version__)
except ImportError:
    print("CrewAI not found. Please install it using the command above.")

**There is functionality of using Microphone to share you problem and text2speech for speaking the output but commented it out and instead used simple text for demonstration purpose**

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
# Step 3: Set up CrewAI agents
from crewai import Agent, LLM
from crewai_tools import RagTool, DOCXSearchTool
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatLiteLLM
import os
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import pathlib
import speech_recognition as sr
import pyttsx3 
llm = LLM(
    model='gemini/gemini-2.0-flash',  # Use provider/model format
    temperature=0.5,
    api_key=os.getenv('GOOGLE_API_KEY')
)

tool = DOCXSearchTool(
    config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model='gemini/gemini-2.0-flash',
                temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/text-embedding-004",
                task_type="retrieval_document",
                title="Embeddings"
            ),
        ),
    )
)

# r = sr.Recognizer() 
# HEADPHONE_MIC_INDEX = 2 

# def SpeakText(command):
#     engine = pyttsx3.init()
#     engine.say(command) 
#     engine.runAndWait()
# r.pause_threshold = 5.0 

# try:
#     # Use specific microphone by index
#     with sr.Microphone(device_index=HEADPHONE_MIC_INDEX) as source2:
#         r.adjust_for_ambient_noise(source2, duration=0.2)
#         audio2 = r.listen(source2,phrase_time_limit=40)
#         MyText = r.recognize_google(audio2)
#         MyText = MyText.lower()
#         print('Did you say:', MyText)
        
# except sr.RequestError as e:
#     print('Could not request results:', e)
    
# except sr.UnknownValueError:
#     print('Could not understand audio')
# Customer_query = MyText

Customer_query = "I am Abhyuday Sharma. My Hp omen Gaming 16 laptop with rtx 3070ti and amd ryzen 7 6800h has display panel issue. Can I know that how much will it cost to fix it? I bought it on 5 Aug 2023 and today is 8 september 2023"
Interactor = Agent(
    role="Interactor",
    goal=f"Interact with the customer {Customer_query} to gather requirements",
    backstory="You’re a friendly and approachable agent who loves to help. Gather information from the customer to understand their needs.",
    llm=llm,
    verbose=True
)

data_collector = Agent(
    role="Document Searcher ",
    goal="From the Interactor output, Understand the customer’s needs and collect relevant data from information in DOCX files",
    backstory="You’re an expert at collecting and organizing data for the given query.",
    llm=llm,
    verbose=True,
    tools=[tool],
)

reply_generator = Agent(
    role="Report Generator",
    goal="Based on the query of customer and the data collected, generate a very polite reply to give back to the customer.",
    backstory="You’re a creative writer who can generate polite and informative replies.",
    llm=llm,
    verbose=True
)

reply_analyser = Agent(
    role="Reply Analyser",
    goal="Analyse the reply generated by the reply_generator and make sure it is polite and informative and summarise it more to 100-200 words",
    backstory="You’re a detail-oriented agent who ensures that replies are up to standard of HP Customer Care.",
    llm=llm,
    verbose=True
)
print("Agents created successfully!")

In [ ]:
# Step 4: Define tasks for the agents
from crewai import Task, Crew


collect_query_task = Task(
    description=f"From the customer, collect the query {Customer_query} and understand their needs.",
    expected_output="A clear and structured understanding of the customer’s needs and query.",
    agent=Interactor
)

dataCollect_task = Task(
    description="Analyze the sales data provided by the Interactor agent.For Identification and required information to be extracted : Search for docs in 'C:/path/to/your/file.docx' . Find necessary costs from document for the things that customer is asking for replacement and warranty of devices",
    expected_output="A detailed output output summary for details if asked, example: warranty related, how much would it cost the customer, etc.",
    agent=data_collector,
    context=[collect_query_task]
)

replygen_task = Task(
    description="Using the data collected, generate a polite reply to the customer. If costumer provides date of purchase and current date, find if the gap comes under warranty and tell the user. you try to provide estimate cost as per documents if replacement cost is asked by customer.",
    expected_output="A polished sales report with actionable recommendations providing user with potential cost and warranty status.",
    agent=reply_generator,
    context = [dataCollect_task]
)
analyse_reply_task = Task(
    description="Analyse the reply and check if the reply meets HP industry standards and it is polite. Summarize the reply into 100-200 words with specific data.",
    expected_output="A polished reply with actionable recommendations within 20-30 words max.",
    agent=reply_analyser,
    context = [replygen_task]
)

crew = Crew(
    agents=[Interactor,data_collector, reply_generator, reply_analyser],
    tasks=[collect_query_task, dataCollect_task, replygen_task, analyse_reply_task],
    verbose=True
)
print("Tasks and crew set up successfully!")

In [ ]:
# Step 5: Run the crew to complete the tasks
result = crew.kickoff()


print("Final Output:\n", result)
# SpeakText(result)